In [1]:
import numpy as np
import pandas as pd
import joblib 
import pickle

In [2]:
# # def slice_and_convert(df):
#     temp = df["saledate"].str.slice(stop = -5) #extracting th date


#     df["saledate"] = pd.to_datetime(temp, 
#                           format = "%m/%d/%y", #format
#                           infer_datetime_format=True) #deal with multiple format

In [3]:
test = pd.read_csv("Test.csv")
test.head()

,SalesID,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,fiModelDesc,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1227829,1006309,3168,121,3,1999,3688.0,Low,5/3/2012 0:00,580G,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1227844,1022817,7271,121,3,1000,28555.0,High,5/10/2012 0:00,936,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
2,1227847,1031560,22805,121,3,2004,6038.0,Medium,5/10/2012 0:00,EC210BLC,...,None or Unspecified,"9' 6""",Manual,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
3,1227848,56204,1269,121,3,2006,8940.0,High,5/10/2012 0:00,330CL,...,None or Unspecified,None or Unspecified,Manual,Yes,Triple,NaN,NaN,NaN,NaN,NaN
4,1227863,1053887,22312,121,3,2005,2286.0,Low,5/10/2012 0:00,650K,...,NaN,NaN,NaN,NaN,NaN,None or Unspecified,PAT,None or Unspecified,NaN,NaN


In [4]:
test_solution = pd.read_csv("random_forest_benchmark_test.csv")
test_solution.head()

,SalesID,SalePrice
0,1227829,13630.0
1,1227844,32530.0
2,1227847,48660.0
3,1227848,96080.0
4,1227863,27440.0


In [5]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer

In [6]:
 

np.random.seed(42)

train_sample = test.sample(frac = 1) #by changing this number you can choose ratio of the sample to train
# # to remove an extra varible form the system to save ram, we will delete the "train" variable
del test

fields_numercial = ["SalesID", "MachineID", 
                             "ModelID", "datasource", # not very imp feature 
                             "auctioneerID", "YearMade"]
#Imputing the numeric data
num_trans = SimpleImputer(strategy='median')
train_sample[fields_numercial] = num_trans.fit_transform(train_sample[fields_numercial])
# num_trans.transform([fields_numerical])

fields_categorical = ["fiProductClassDesc", "state", "fiBaseModel", 
                      "ProductGroup", "Hydraulics", # not very imp feature
                      "ProductGroupDesc", "Enclosure"]

# #Imputing the categorical data
cat_trans = SimpleImputer(strategy='constant')
train_sample[fields_categorical] = cat_trans.fit_transform(train_sample[fields_categorical])
# cat_trans.transform([fields_categorical])

cat_enc = OrdinalEncoder()
train_sample[fields_categorical] = cat_enc.fit_transform(train_sample[fields_categorical])

In [7]:
train_sample.head()

,SalesID,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,fiModelDesc,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
396,1229148.0,1042578.0,9579.0,121.0,3.0,2004.0,3290.0,Medium,6/15/2012 0:00,S250,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
247,1228669.0,1055336.0,3537.0,121.0,3.0,1000.0,7986.0,Medium,5/17/2012 0:00,416B,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9260,6452395.0,2506026.0,25020.0,149.0,13.0,2000.0,NaN,NaN,9/22/2012 0:00,555E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1430,4319441.0,2304860.0,4603.0,172.0,1.0,1991.0,5689.0,Low,5/21/2012 0:00,310D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10328,6624176.0,2552495.0,3567.0,149.0,1.0,1994.0,NaN,NaN,9/25/2012 0:00,446B,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
np.random.seed(42)

test_sample_X = train_sample[fields_numercial + fields_categorical]
test_sample_X.head()

,SalesID,MachineID,ModelID,datasource,auctioneerID,YearMade,fiProductClassDesc,state,fiBaseModel,ProductGroup,Hydraulics,ProductGroupDesc,Enclosure
396,1229148.0,1042578.0,9579.0,121.0,3.0,2004.0,38.0,23.0,675.0,2.0,3.0,2.0,0.0
247,1228669.0,1055336.0,3537.0,121.0,3.0,1000.0,1.0,23.0,134.0,0.0,11.0,0.0,0.0
9260,6452395.0,2506026.0,25020.0,149.0,13.0,2000.0,4.0,41.0,173.0,0.0,11.0,0.0,2.0
1430,4319441.0,2304860.0,4603.0,172.0,1.0,1991.0,1.0,11.0,94.0,0.0,11.0,0.0,0.0
10328,6624176.0,2552495.0,3567.0,149.0,1.0,1994.0,3.0,42.0,146.0,0.0,11.0,0.0,2.0


In [10]:
test_sample_X.to_csv("Sample_data_encoded.csv", index = False)

In [9]:
# best_parameter = {'n_estimators': 1200,
#  'min_samples_split': 2,
#  'min_samples_leaf': 2,
#  'max_features': 'sqrt',
#  'max_depth': 50,
#  'bootstrap': False}
# np.random.seed(42)

# test_sample_X = train_sample[fields_categorical + fields_numercial]

# train_sample_y = train_sample["SalePrice"]

# #to save the space (RAM)
# del train_sample

# # not requied in RandomSearchCV
# sample_train_X, sample_test_X, \
# sample_train_y, sample_test_y = train_test_split(train_sample_X, train_sample_y, 
#                                                  test_size = 0.2)  
# #to save spavve (RAM) we will delete
# del train_sample_X, train_sample_y

In [1]:
# the models are more than 5GB each. Depending on our requirement. We might(not) train them.

# loaded_pickle_model = pickle.load(open('RandomForestPickle.pkl', "rb"))
# pickle_y_preds = loaded_pickle_model.predict(test_sample_X)
# pickle_y_preds.head()

# loaded_joblib_model = joblib.load(filename = 'RandomForestJoblib.joblib')
# joblib_y_preds = loaded_joblib_model.predict(test_sample_X)
# joblib_y_preds.head()